In [ ]:
# The following command can be copy-pasted to CLI but it does not really download a catalogue
# Currently it is not used
java -jar stilts.jar tapquery \
  tapurl="http://tapvizier.cds.unistra.fr/TAPVizieR/tap" \
  adql='SELECT * FROM "VIII/92/first14"' \
  ofmt=fits \
  out=./first14.fits

In [ ]:
#For step3 cooking on one source, examples

# The following codes are for single catalogue, just for reference
# # 2.  After filtering, adjust the groupIDs accordingly
# java -jar $stilts_dir tgroup in="filtered_r1" \
#     keys=GroupID \
#     aggcols="0;count;GroupCount" \
#     out=group_count.fits
# java -jar $stilts_dir tpipe in=group_count.fits \
#     cmd='select "GroupCount == 1"' \
#     out=singleton_groupids.fits

# java -jar $stilts_dir tmatch2 \
#     in1=filtered_r1.fits \
#     in2=singleton_groupids.fits \
#     matcher=exact \
#     join=all1 \
#     values1=GroupID values2=GroupID \
#     out=temp_with_gcount.fits

# java -jar $stilts_dir tpipe \
#     in=temp_with_gcount.fits \
#     cmd='addcol GroupID "!NULL_GroupID_1 ? GroupID_1 : NULL"' \
#     cmd='delcols GroupID_2' \
#     cmd='delcols GroupID_1' \
#     out=temp_with_gcount.fits
# exit

# #Note, if you don't specify the !NULL option, stilts will treat the ELSE condition (when GroupCount is NULL) as NULL, so it will give GroupSize = NULL rather than keeping the original GroupSize value.
# rpl_cmd='replacecol GroupSize "!NULL_GroupCount && GroupCount == 1 ? 1 : GroupSize"'
# # This may overlappipng with the previous command, but it is necessary to ensure that GroupSize is set to 1 for singleton groups.
# rpl_cmd2='replaceval NULL 1 "GroupSize"'
# java -jar $stilts_dir tpipe ifmt=fits \
#     in=temp_with_gcount.fits \
#     cmd="$rpl_cmd" \
#     cmd="$rpl_cmd2" \
#     out=filtered_r1_CLN.fits

# # 3. Flag the high-z and single matched sources as tier 1

# flag_cmd1='addcol source_tier "GroupSize==1 && Z_o1>=3 ? 1 : 0"'
# # flag_cmd2='replacecol source_tier "source_tier == 0 && group_z_min > 3 ? 2 : 0"'
# # flag_cmd3='replacecol source_tier "source_tier == 0 && group_z_min <= 3 && Z_o1>=3 ? 3 : 0"'
# java -jar $stilts_dir tpipe ifmt=fits in=filtered_r1_CLN.fits \
#     cmd="$flag_cmd1" \
#     out=mr1_t1.fits

# cook the grouped sources
# java -jar $stilts_dir tgroup \
#     in=filtered_r3.fits \
#     keys=GroupID \
#     aggcols='Z_o1;max;group_z_max Z_o1;min;group_z_min 0;count;GroupSize' \
#     out=group_zcheck.fits

# java -jar $stilts_dir tpipe \
#     in=group_zcheck.fits \
#     cmd='select "!NULL_GroupID && GroupSize > 1 && group_z_max > 3"' \
#     cmd='keepcols "GroupID group_z_min"' \
#     out=r3_group_hz_sources.fits

# java -jar $stilts_dir tmatch2 \
#     in1=filtered_r3.fits \
#     in2=r3_group_hz_sources.fits \
#     matcher=exact \
#     join=all1 \
#     find=all \
#     values1=GroupID values2=GroupID \
#     ocmd='delcols "GroupID GroupSize"' \
#     ocmd='addcol GroupID "!NULL_GroupID_1 ? GroupID_1 : NULL"' \
#     ocmd='addcol GroupSize "!NULL_GroupSize_1 ? GroupSize_1 : NULL"' \
#     ocmd='delcols "GroupID_1 GroupID_2 GroupSize_1"' \
#     out=filtered_r3_Tier23.fits

#         #    cmd='replacecol source_tier (
#         # source_tier < 2 && GroupID_NULL ? source_tier :
#         # (source_tier < 2 && z_min > 3 ? 3 :
#         # (source_tier < 2 ? 2 : source_tier)))' \
# exit

In [9]:
import pandas as pd
from astropy.io import fits
import numpy as np
import os
from astropy.table import Table
from astropy.coordinates import SkyCoord

In [10]:

sdss_galaxy = "../catalogues/dr16_galaxy.fits"
dr16q_short = "tmp_sdss.fits"
outname = "tmp_sdss16_comb.fits"

with fits.open(sdss_galaxy) as hdul:
    data = hdul[1].data 
    table_g = Table(data)  
df_galaxy = table_g.to_pandas()  


In [7]:
df_galaxy



,specobjid(d),SPECOBJID,PLUG_RA,PLUG_DEC,CLASS,SUBCLASS,Z,Z_ERR,RCHI2,ZWARNING,Z_PERSON,CLASS_PERSON,Z_CONF_PERSON,COMMENTS_PERSON
0,2.994897e+17,299489676975171584,146.71421,-1.041304,GALAXY,,0.021275,0.000009,1.074070,0,0.0,0,0,
1,2.994900e+17,299489951853078528,146.91945,-0.990492,GALAXY,,0.213925,0.000052,1.331336,0,0.0,0,0,
2,2.994905e+17,299490501608892416,146.85983,-0.808902,GALAXY,,0.126554,0.000062,1.338671,0,0.0,0,0,
3,2.994908e+17,299490776486799360,146.76339,-0.810433,GALAXY,,0.448645,0.000168,1.435746,0,0.0,0,0,
4,2.994911e+17,299491051364706304,146.96390,-0.545003,GALAXY,STARBURST,0.055988,0.000011,1.315894,0,0.0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863630,1.084494e+19,10844941542625660928,141.89192,28.041277,GALAXY,,1.037471,0.000178,1.032615,0,0.0,0,0,
2863631,1.084494e+19,10844941817503567872,141.83234,28.014509,GALAXY,,0.742518,0.000056,1.076094,0,0.0,0,0,
2863632,1.084494e+19,10844942367259381760,141.92392,28.090031,GALAXY,,1.056773,0.000149,1.094800,0,0.0,0,0,
2863633,1.084494e+19,10844942642137288704,141.97961,27.957674,GALAXY,,1.012589,0.000177,1.138485,0,0.0,0,0,


In [24]:
source_coords = SkyCoord(ra=df_galaxy['PLUG_RA'][:5], dec=df_galaxy['PLUG_DEC'][:5], unit='deg')
# make a sdss style source name based on RA and DEC
ra = source_coords.ra.to_string(unit='hour', sep=':', precision=2, pad=True)
dec = source_coords.dec.to_string(unit='deg', sep=':', precision=1, pad=True)
sdss_name = np.char.add('g', np.char.add(np.char.replace(ra, ':', ''), np.char.replace(dec, ':', '')))
sdss_name

array(['g094651.41-010228.7', 'g094740.67-005925.8',
       'g094726.36-004832.0', 'g094703.21-004837.6',
       'g094751.34-003242.0'], dtype='<U19')